In [1]:
import pandas as pd
df=pd.read_csv('groups_apt.csv')

In [2]:
df

,Unnamed: 0,Name,Alias,description
0,0,admin@338,NaN,admin@338 is a China-based cyber threat group....
1,1,APT1,"Comment Crew, Comment Group, Comment Panda",APT1 is a Chinese threat group that has been a...
2,2,APT12,"IXESHE, DynCalc, Numbered Panda, DNSCALC",APT12 is a threat group that has been attribut...
3,3,APT16,NaN,APT16 is a China-based threat group that has l...
4,4,APT17,Deputy Dog,APT17 is a China-based threat group that has c...
...,...,...,...,...
89,89,Thrip,NaN,Thrip is an espionage group that has targeted ...
90,90,Tropic Trooper,KeyBoy,Tropic Trooper is an unaffiliated threat group...
91,91,Turla,"Waterbug, WhiteBear, VENOMOUS BEAR, Snake, Kry...",Turla is a Russian-based threat group that has...
92,92,Winnti Group,Blackfly,Winnti Group is a threat group with Chinese or...


In [15]:
text=df['description'][22]
print(text)

Cleaver is a threat group that has been attributed to Iranian actors and is responsible for activity tracked as Operation Cleaver.  Strong circumstantial evidence suggests Cleaver is linked to Threat Group 2889 (TG-2889).


In [3]:
words=['Dust Storm','South Korea','Japan','United States','Asian ']
label=['group','target_country','target_country','target_country','target_country']

In [4]:
train=[]
entities=[]
if len(words)==len(label):
    for i in range(len(words)):
        word=text.find(words[i])
        print(word)
        if i==0:
            entities.append('[('+str(word)+','+str(word+len(words[i]))+','+'"'+label[i]+'"'+')')
        elif i==(len(words)-1):
            entities.append('('+str(word)+','+str(word+len(words[i]))+','+'"'+label[i]+'"'+')]}')
        else:
            entities.append('('+str(word)+','+str(word+len(words[i]))+','+'"'+label[i]+'"'+')')
        
    entities=','.join(entities)
    train.append('("'+text+'"'+','+'{"entities":'+entities+')')
else:
    print('once check')
    print("words:",len(words))
    print("labels:",len(label))

0
77
70
94
139


In [5]:
train[0]

'("Dust Storm is a threat group that has targeted multiple industries in Japan, South Korea, the United States, Europe, and several Southeast Asian countries.",{"entities":[(0,10,"group"),(77,88,"target_country"),(70,75,"target_country"),(94,107,"target_country"),(139,145,"target_country")]})'

In [3]:
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

In [4]:
TRAIN_DATA=[
    ("admin@338 is a China-based cyber threat group. It has previously used newsworthy events as lures to deliver malware and has primarily targeted organizations involved in financial, economic, and trade policy, typically using publicly available RATs such as PoisonIvy, as well as some non-public backdoors.",{"entities":[(0,9,"group"),(15,26,"threat_actor"),(169,178,"target"),(180,188,"target"),(194,206,"target"),(256,265,"path")]}),
    ("APT1 is a Chinese threat group that has been attributed to the 2nd Bureau of the People’s Liberation Army (PLA) General Staff Department’s (GSD) 3rd Department, commonly known by its Military Unit Cover Designator (MUCD) as Unit 61398.",{"entities":[(0,4,"group"),(10,18,"threat_actor"),(63,105,"target")]}),
    ("APT12 is a threat group that has been attributed to China. The group has targeted a variety of victims including but not limited to media outlets, high-tech companies, and multiple governments.",{"entities":[(0,5,"group"),(52,57,"threat_actor"),(132,145,"target"),(147,166,"target"),(172,192,"target")]}),
    ("APT16 is a China-based threat group that has launched spearphishing campaigns targeting Japanese and Taiwanese organizations.",{"entities":[(0,5,"group"),(11,23,"threat_actor"),(88,96,"target"),(101,110,"target")]}),
    ("APT17 is a China-based threat group that has conducted network intrusions against U.S. government entities, the defense industry, law firms, information technology companies, mining companies, and non-government organizations.",{"entities":[(0,5,"group"),(11,22,"threat_actor"),(55,73,"path"),(82,86,"target_country"),(112,128,"target"),(130,139,"target"),(141,163,"target"),(175,181,"target"),(197,211,"target")]}),
    ("APT18 is a threat group that has operated since at least 2009 and has targeted a range of industries, including technology, manufacturing, human rights groups, government, and medical.",{"entities":[(0,5,"group"),(57,61,"date"),(112,122,"target"),(124,137,"target"),(139,151,"target"),(160,170,"target"),(176,183,"target")]}),
    ("APT19 is a Chinese-based threat group that has targeted a variety of industries, including defense, finance, energy, pharmaceutical, telecommunications, high tech, education, manufacturing, and legal services. In 2017, a phishing campaign was used to target seven law and investment firms.  Some analysts track APT19 and Deep Panda as the same group, but it is unclear from open source information if the groups are the same.",{"entities":[(0,5,"group"),(11,24,"threat_group"),(321,331,"same_group"),(175,188,"target"),(91,98,"target"),(133,151,"target"),(117,131,"target")]}),
    ("APT28 is a threat group that has been attributed to Russia\'s Main Intelligence Directorate of the Russian General Staff by a July 2018 U.S. Department of Justice indictment. This group reportedly compromised the Hillary Clinton campaign, the Democratic National Committee, and the Democratic Congressional Campaign Committee in 2016 in an attempt to interfere with the U.S. presidential election. APT28 has been active since at least 2004.",{"entities":[(0,5,"group"),(52,58,"threat_group"),(434,438,"date"),(328,332,"date"),(212,236,"target"),(242,261,"target"),(281,305,"target")]}),
    ("APT29 is threat group that has been attributed to the Russian government and has operated since at least 2008.   This group reportedly compromised the Democratic National Committee starting in the summer of 2015.",{"entities":[(0,5,"group"),(54,61,"threat_group"),(105,109,"date"),(207,211,"date"),(151,170,"target")]}),
    ("APT3 is a China-based threat group that researchers have attributed to China\'s Ministry of State Security.   This group is responsible for the campaigns known as Operation Clandestine Fox, Operation Clandestine Wolf, and Operation Double Tap.   As of June 2015, the group appears to have shifted from targeting primarily US victims to primarily political organizations in Hong Kong. \nMITRE has also developed an APT3 Adversary Emulation Plan.",{"entities":[(0,4,"group"),(10,21,"threat_group"),(372,381,"target_country"),(256,260,"date"),(321,331,"target")]}),
    ("APT30 is a threat group suspected to be associated with the Chinese government.  While Naikon shares some characteristics with APT30, the two groups do not appear to be exact matches.",{"entities":[(0,5,"group"),(60,67,"threat_group")]}),
    ("Dragonfly 2.0 is a suspected Russian group that has targeted government entities and multiple U.S. critical infrastructure sectors since at least March 2016.   There is debate over the extent of overlap between Dragonfly 2.0 and Dragonfly, but there is sufficient evidence to lead to these being tracked as two separate groups.",{"entities":[(0,13,"group"),(29,37,"threat_group"),(94,98,"target_country"),(146,156,"date")]}),
    ("DragonOK is a threat group that has targeted Japanese organizations with phishing emails. Due to overlapping TTPs, including similar custom tools, DragonOK is thought to have a direct or indirect relationship with the threat group Moafee.  It is known to use a variety of malware, including Sysget/HelloBridge, PlugX, PoisonIvy, FormerFirstRat, NFlog, and NewCT.",{"entities":[(0,9,"group"),(45,54,"target_country"),(231,237,"same_group")]}),
    ("Dust Storm is a threat group that has targeted multiple industries in Japan, South Korea, the United States, Europe, and several Southeast Asian countries.",{"entities":[(0,10,"group"),(77,88,"target_country"),(70,75,"target_country"),(94,107,"target_country"),(139,145,"target_country")]}),]

In [5]:
model=None
if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe("ner")
    
for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

Created blank 'en' model


In [6]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]

In [8]:
with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(100):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

Losses {'ner': 50.04646199790709}
Losses {'ner': 46.9870311577273}
Losses {'ner': 36.58805302936923}
Losses {'ner': 67.7979691387506}
Losses {'ner': 42.9267841148063}
Losses {'ner': 31.154274665798575}
Losses {'ner': 26.853744176002063}
Losses {'ner': 32.86801939710722}
Losses {'ner': 28.025471247285594}
Losses {'ner': 36.31076856415828}
Losses {'ner': 24.971844998096632}
Losses {'ner': 25.633978552147212}
Losses {'ner': 25.287065743270905}
Losses {'ner': 18.883270231655914}
Losses {'ner': 44.03638009799452}
Losses {'ner': 44.393577775142255}
Losses {'ner': 36.770851376199424}
Losses {'ner': 27.13435382468843}
Losses {'ner': 28.35231408366001}
Losses {'ner': 31.041384184712996}
Losses {'ner': 15.607410032112007}
Losses {'ner': 22.47426059259185}
Losses {'ner': 15.91419248589861}
Losses {'ner': 16.310649256336887}
Losses {'ner': 19.142589952654788}
Losses {'ner': 16.98496905159933}
Losses {'ner': 20.58677222927684}
Losses {'ner': 50.91073337226517}
Losses {'ner': 16.76425870989591}
Loss

In [10]:
for text, _ in TRAIN_DATA:
        doc = nlp(text)
        #print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Tokens [('APT3', 'group', 3), ('is', '', 2), ('a', '', 2), ('China', 'threat_group', 3), ('-', 'threat_group', 1), ('based', 'threat_group', 1), ('threat', '', 2), ('group', '', 2), ('that', '', 2), ('researchers', '', 2), ('have', '', 2), ('attributed', '', 2), ('to', '', 2), ('China', '', 2), ("'s", '', 2), ('Ministry', '', 2), ('of', '', 2), ('State', '', 2), ('Security', '', 2), ('.', '', 2), ('  ', '', 2), ('This', '', 2), ('group', '', 2), ('is', '', 2), ('responsible', '', 2), ('for', '', 2), ('the', '', 2), ('campaigns', '', 2), ('known', '', 2), ('as', '', 2), ('Operation', '', 2), ('Clandestine', '', 2), ('Fox', '', 2), (',', '', 2), ('Operation', '', 2), ('Clandestine', '', 2), ('Wolf', '', 2), (',', '', 2), ('and', '', 2), ('Operation', '', 2), ('Double', '', 2), ('Tap', '', 2), ('.', '', 2), ('  ', '', 2), ('As', '', 2), ('of', '', 2), ('June', '', 2), ('2015', 'date', 3), (',', '', 2), ('the', '', 2), ('group', '', 2), ('appears', '', 2), ('to', '', 2), ('have', '', 2), (

Tokens [('Dust', 'group', 3), ('Storm', 'group', 1), ('is', '', 2), ('a', '', 2), ('threat', '', 2), ('group', '', 2), ('that', '', 2), ('has', '', 2), ('targeted', '', 2), ('multiple', '', 2), ('industries', '', 2), ('in', '', 2), ('Japan', 'target_country', 3), (',', '', 2), ('South', 'target_country', 3), ('Korea', 'target_country', 1), (',', '', 2), ('the', '', 2), ('United', 'target_country', 3), ('States', 'target_country', 1), (',', '', 2), ('Europe', '', 2), (',', '', 2), ('and', '', 2), ('several', '', 2), ('Southeast', '', 2), ('Asian', '', 2), ('countries', '', 2), ('.', '', 2)]
Tokens [('Dragonfly', 'group', 3), ('2.0', 'group', 1), ('is', '', 2), ('a', '', 2), ('suspected', '', 2), ('Russian', '', 2), ('group', '', 2), ('that', '', 2), ('has', '', 2), ('targeted', '', 2), ('government', '', 2), ('entities', '', 2), ('and', '', 2), ('multiple', '', 2), ('U.S.', 'target_country', 3), ('critical', '', 2), ('infrastructure', '', 2), ('sectors', '', 2), ('since', '', 2), ('at',

In [16]:
doc=nlp('Cleaver is a threat group that has been attributed to Iranian actors and is responsible for activity tracked as Operation Cleaver.  Strong circumstantial evidence suggests Cleaver is linked to Threat Group 2889 (TG-2889)')
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
#print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('Cleaver', 'group'), ('Strong', 'threat_group')]


In [203]:
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding


# training data
TRAIN_DATA = [
    ("Who is Shaka Khan?", {"entities": [(7, 17, "PERSON")]}),
    ("I like London and Berlin.", {"entities": [(7, 13, "LOC"), (18, 24, "LOC")]}),
]


@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    output_dir=("Optional output directory", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int),
)
def main(model=None, output_dir=None, n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

    # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

In [204]:
if __name__ == "__main__":
    plac.call(main)

SystemExit: 2

In [179]:
TRAIN_DATA_1 = [
    ("Who is Shaka Khan?", {"entities": [(7, 17, "PERSON")]}),
    ("I like London and Berlin.", {"entities": [(7, 13, "LOC"), (18, 24, "LOC")]}),
]

In [176]:
for _,annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
            ner.add_label(ent[2])